# 01 - BaT Scraping

Scrape completed Porsche 911 auctions from Bring a Trailer.

**Outputs:**
- `data/raw/bat_listings.parquet` - Raw scraped data

In [ ]:
import logging
import pandas as pd
from pathlib import Path

from price_analysis.constants import SCRAPING_DEFAULTS
from price_analysis.scraping import fetch_auctions, validate_scraped_data, DataQualityError
from price_analysis.scraping.bat import listings_to_dataframe

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Paths
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = RAW_DIR / "bat_listings.parquet"

## Configure Scraping

Adjust search queries and pagination as needed.

In [ ]:
# Search queries - multiple generation-specific searches to get more results
# BaT seems to limit how many results a single search returns
QUERIES = [
    # "Porsche 996",  # 1999-2004 (996.1 and 996.2)
    # "Porsche 997",  # 2005-2012 (997.1 and 997.2)
    "Porsche 991",  # 2012-2019 (991.1 and 991.2)
    "Porsche 992",  # 2020+ (992.1 and 992.2)
    "Porsche 911",  # Catch air-cooled + anything else missed
]

# Scraping parameters (from centralized constants)
MAX_CLICKS = SCRAPING_DEFAULTS["max_clicks"]
DELAY = SCRAPING_DEFAULTS["delay"]
HEADLESS = SCRAPING_DEFAULTS["headless"]

## Debug: Inspect BaT Page Structure (Optional)

Skip this section - fixture files already saved to `tests/fixtures/`.
Only re-run if BaT changes their DOM structure.

In [ ]:
# Debug: Save sample pages for selector inspection
import time
from price_analysis.scraping import create_driver, save_debug_page

# Create driver (set HEADLESS=False above to watch)
driver = create_driver(headless=HEADLESS)

try:
    # 1. Fetch search results page
    search_url = "https://bringatrailer.com/auctions/results/?s=Porsche+911"
    logger.info(f"Fetching search page: {search_url}")
    driver.get(search_url)
    time.sleep(3)  # Wait for JS to load
    save_debug_page(driver, "bat_search_page", output_dir="../tests/fixtures")

    # 2. Fetch a specific listing page (992.1)
    listing_url = "https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/"
    logger.info(f"Fetching listing: {listing_url}")
    driver.get(listing_url)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_992", output_dir="../tests/fixtures")

    # 3. Fetch another listing (997.2) for variety
    listing_url_997 = "https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/"
    logger.info(f"Fetching listing: {listing_url_997}")
    driver.get(listing_url_997)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_997", output_dir="../tests/fixtures")

    logger.info("Debug pages saved to tests/fixtures/")

finally:
    driver.quit()

## Run Scraper

This will take a while depending on MAX_PAGES. Each page + listing takes ~3-5 seconds.

In [ ]:
# Load existing URLs for incremental scraping (skip already-fetched listings)
existing_urls: set[str] = set()
if OUTPUT_PATH.exists():
    existing_df = pd.read_parquet(OUTPUT_PATH)
    existing_urls = set(existing_df["listing_url"])
    logger.info(f"Loaded {len(existing_urls)} existing URLs - will skip these")

all_listings = []

for query in QUERIES:
    logger.info(f"Scraping: {query}")
    listings = fetch_auctions(
        query=query,
        max_clicks=MAX_CLICKS,
        delay=DELAY,
        headless=HEADLESS,
        existing_urls=existing_urls,
    )
    all_listings.extend(listings)
    logger.info(f"Found {len(listings)} NEW listings for '{query}'")

logger.info(f"Total NEW listings scraped: {len(all_listings)}")

## Convert to DataFrame and Save

In [ ]:
df = listings_to_dataframe(all_listings)

# Run data quality checks and filter low-price listings (<$10k)
# Returns filtered DataFrame, raises DataQualityError if checks fail
df = validate_scraped_data(df)

display(df.head(10))
print(f"\nShape: {df.shape}")

In [ ]:
# Append to existing if present, otherwise create new
if OUTPUT_PATH.exists():
    existing = pd.read_parquet(OUTPUT_PATH)
    df = pd.concat([existing, df], ignore_index=True)
    df = df.drop_duplicates(subset=["listing_url"], keep="last")
    logger.info(f"Merged with existing data: {len(df)} total listings")

df.to_parquet(OUTPUT_PATH, index=False)
logger.info(f"Saved to {OUTPUT_PATH}")

## Quick Inspection

In [ ]:
print("Counts by generation:")
display(df["generation"].value_counts(dropna=False))

In [ ]:
print("Counts by trim:")
display(df["trim"].value_counts(dropna=False))

In [ ]:
print("Counts by transmission:")
display(df["transmission"].value_counts(dropna=False))

In [ ]:
# Check parsing quality - how many have all required fields?
required = ["sale_price", "model_year", "generation", "trim", "transmission", "mileage"]
complete = df[required].notna().all(axis=1).sum()
print(
    f"\nListings with all required fields: {complete} / {len(df)} ({complete / len(df) * 100:.1f}%)"
)

In [ ]:
# Sample some listings to verify parsing
print("Sample listings for manual verification:")
sample = df.sample(min(10, len(df)), random_state=42)
for _, row in sample.iterrows():
    print(f"\n{row['title_raw']}")
    print(
        f"  Parsed: {row['model_year']} {row['generation']} {row['trim']} ({row['transmission']})"
    )
    print(f"  Price: ${row['sale_price']:,}" if pd.notna(row["sale_price"]) else "  Price: N/A")
    print(f"  Mileage: {row['mileage']:,}" if pd.notna(row["mileage"]) else "  Mileage: N/A")